In [15]:
import random
import numpy as np
import pandas as pd

from copy import deepcopy

In [23]:
vale13 = pd.read_excel('data/raw/vale13FEELS.xlsx')
vale14 = pd.read_excel('data/raw/vale14FEELS.xlsx')
vale15 = pd.read_excel('data/raw/vale15FEELS.xlsx')
vale16 = pd.read_excel('data/raw/vale15FEELS.xlsx')
vale17_1 = pd.read_excel('data/raw/vale17-1FEELS.xlsx')
vale17_2 = pd.read_excel('data/raw/vale17-2FEELS.xlsx')

In [24]:
for i in vale13['Tweets'][:20]:
    print(i)

Você já pode saber mais sobre as nossas operações na Nova Caledônia, na China e no Japão. Acesse http://t.co/iujnLI0rU4 e escolha o país!
Fizemos bonito no ranking @WomeninMining, que aponta as 100 mulheres mais inspiradoras da indústria mineral. Uhuuu! http://t.co/CUmFRL7haA
Não deixe para depois o que você pode mudar em 2014. Escolha uma das nossas resoluções e compartilhe com um amigo! http://t.co/MJFgpJJI0N
@gustavoobh Obrigado pelas sugestões. =)
Paragominas é um exemplo de como aliar desenvolvimento econômico e proteção ambiental. Veja o que o Fundo Vale fez! http://t.co/sHMK7Lm1lq
Estamos arrecadando doações p/ as vítimas das enchentes do ES e MG. Para cada R$ 1 doado, doaremos o dobro. Participe http://t.co/8kO98yeAr5
Pode deixar, @gustavoobh. Vamos enviar a sugestão para a área responsável. Havendo possibilidade, entraremos em contato com a prefeitura =)
Oi, @gustavoobh. Através de patrocínio?
Ela está funcionando com restrições, @juliana_zc. Para mais informações, por favor, 

In [25]:
vale13_17 = vale13.append(vale14, ignore_index = True)
vale13_17 = vale13_17.append(vale15, ignore_index = True)
vale13_17 = vale13_17.append(vale16, ignore_index = True)
vale13_17 = vale13_17.append(vale17_1, ignore_index = True)
vale13_17 = vale13_17.append(vale17_2, ignore_index = True)

In [26]:
vale13_17.shape

(4809, 4)

In [27]:
vale13_17.head()

,Data,Tweets,Polaridade,Subjetividade
0,2013-12-30 18:24:01,Você já pode saber mais sobre as nossas operaç...,0.335227,0.477273
1,2013-12-30 12:55:39,"Fizemos bonito no ranking @WomeninMining, que ...",0.458333,0.833333
2,2013-12-27 19:42:52,Não deixe para depois o que você pode mudar em...,0.000000,0.000000
3,2013-12-27 16:54:40,@gustavoobh Obrigado pelas sugestões. =),0.350000,0.600000
4,2013-12-27 13:42:28,Paragominas é um exemplo de como aliar desenvo...,0.250000,0.200000


#### Opção 1

In [28]:
option1 = deepcopy(vale13_17)
option1.Subjetividade += 1

option1 = option1.assign(Sentimento = lambda x: x['Polaridade'] / x['Subjetividade'])
option1["Sentimento"].fillna(0, inplace = True) 
option1.Data = option1.Data.apply(lambda x: x.date().strftime("%Y%m%d"))
tweetsCount = option1.groupby('Data').count().Polaridade.to_frame().reset_index()
tweetsCount.columns = ['Data', 'Quantidade']
option1 = option1.groupby('Data').sum().reset_index()
option1 = option1.merge(tweetsCount, on = 'Data')

In [29]:
option1.head()

,Data,Polaridade,Subjetividade,Sentimento,Quantidade
0,20130102,2.500000,7.333333,1.312500,4
1,20130103,0.000000,1.000000,0.000000,1
2,20130107,0.236364,3.054545,0.156250,2
3,20130108,-0.100000,2.100000,-0.090909,2
4,20130109,1.800000,4.000000,0.900000,2


In [37]:
option1.to_csv('data/processed/sentiment_op1.csv', index = False)

#### Opcao 2

In [30]:
def categorizeValue(value):
    compare = int(value * 10)
    if compare > 2:
        return 'Bom'
    elif compare < -2:
        return 'Ruim'
    else:
        return 'Neutro'

In [31]:
option2 = deepcopy(vale13_17)
option2.Subjetividade += 1

option2 = option2.assign(Sentimento = lambda x: x['Polaridade'] / x['Subjetividade'])
option2["Sentimento"].fillna(0, inplace = True) 
option2.Data = option2.Data.apply(lambda x: x.date().strftime("%Y%m%d"))

In [32]:
option2['Sentimento'] = option2['Sentimento'].apply(lambda x: categorizeValue(x))

In [33]:
option2.head()

,Data,Tweets,Polaridade,Subjetividade,Sentimento
0,20131230,Você já pode saber mais sobre as nossas operaç...,0.335227,1.477273,Neutro
1,20131230,"Fizemos bonito no ranking @WomeninMining, que ...",0.458333,1.833333,Neutro
2,20131227,Não deixe para depois o que você pode mudar em...,0.000000,1.000000,Neutro
3,20131227,@gustavoobh Obrigado pelas sugestões. =),0.350000,1.600000,Neutro
4,20131227,Paragominas é um exemplo de como aliar desenvo...,0.250000,1.200000,Neutro


In [34]:
option2 = pd.get_dummies(option2, columns = ['Sentimento'])
option2['Quantidade'] = option2['Sentimento_Bom']+ option2['Sentimento_Neutro']+ option2['Sentimento_Ruim']
option2.head()

,Data,Tweets,Polaridade,Subjetividade,Sentimento_Bom,Sentimento_Neutro,Sentimento_Ruim,Quantidade
0,20131230,Você já pode saber mais sobre as nossas operaç...,0.335227,1.477273,0,1,0,1
1,20131230,"Fizemos bonito no ranking @WomeninMining, que ...",0.458333,1.833333,0,1,0,1
2,20131227,Não deixe para depois o que você pode mudar em...,0.000000,1.000000,0,1,0,1
3,20131227,@gustavoobh Obrigado pelas sugestões. =),0.350000,1.600000,0,1,0,1
4,20131227,Paragominas é um exemplo de como aliar desenvo...,0.250000,1.200000,0,1,0,1


In [35]:
option2 = option2.groupby('Data').sum().sort_values(by='Sentimento_Bom', ascending = False).reset_index()

In [36]:
option2.head()

,Data,Polaridade,Subjetividade,Sentimento_Bom,Sentimento_Neutro,Sentimento_Ruim,Quantidade
0,20130909,10.125595,35.408929,10,10,0,20
1,20151204,8.222944,181.651732,10,132,2,144
2,20131010,10.139289,43.666335,9,19,0,28
3,20150211,3.554762,57.039286,8,32,0,40
4,20171220,5.451705,25.470265,7,12,0,19


In [52]:
option2.to_csv('data/processed/sentiment_op2.csv', index = False)

#### Opção 3

In [39]:
option3 = deepcopy(option1)
option3.drop(columns = ['Sentimento'], inplace = True)

In [57]:
option3.to_csv('data/processed/sentiment_op3.csv', index = False)